In [151]:
import rasterio
from pathlib import Path 
import geopandas as gpd
import pandas as pd
import numpy as np

# 0. Data Check
## treepoint data list
- manup_treepoint.shp
- mandown_treepoint.shp
- br_rest1_treepoint.shp
- br_rest2_treepoint.shp

In [152]:
# data load
tp_up = gpd.GeoDataFrame.from_file("tree_points/manup_treepoint.shp")
tp_down = gpd.GeoDataFrame.from_file("tree_points/mandown_treepoint.shp")
tp_br1 = gpd.GeoDataFrame.from_file("tree_points/br_rest1_treepoint.shp")
tp_br2 = gpd.GeoDataFrame.from_file("tree_points/br_rest2_treepoint.shp")
tp_br = gpd.GeoDataFrame( pd.concat( [tp_br1, tp_br2], ignore_index=True) )

In [153]:
# subclass change function
def sc_change(tp):
    # replace with *
    tp_sc = tp['subclass'].fillna('*')
    
    # subclass string to int
    tp_sc = tp_sc.to_frame()
    sc_list = {'subclass':{'Rosidae':'1', 'Hamamelididae':'2', 'Asteridae':'3',
                           'Dilleniidae':'4', 'Magnoliidae':'5', '*':'6'}}
    tp_sc = tp_sc.replace(sc_list)
    
    # change
    tp.drop(["subclass", "GenusSpeci", "OBJECTID", "Lat", "Long"], axis = 1, inplace=True)
    tp = pd.concat([tp, tp_sc], axis=1)
    
    return tp

In [154]:
# run
up = sc_change(tp_up)
down = sc_change(tp_down)
br = sc_change(tp_br)

In [ ]:
## result: ['geo','subclass'(number)] || up(28273,2) down(24128,2) br(33398,2)

# 1. Point Sampling
- NDVI point smapling and Delete under 0.2
- Point Sampling

In [158]:
for i in ['NDVI', 'SBI', 'GVI', 'YVI', 'WBI']:
    for j in ['up', 'down', 'br']:
        globals()[f'{i}{j}_file'] = '../KOMPSAT 위성영상/데이터 전처리/4. data/'+j+'/'+i+'.tif'
        globals()[f'{i}{j}_src'] = rasterio.open(globals()[f'{i}{j}_file'])

In [159]:
# NDVI point smapling and Delete under 0.2
def NDVI02(tp, file, src):
    coord_list = [(x,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)] 
    tp[Path(file).stem] = [x for x in src.sample(coord_list)] # 포인트 샘플링
    tp[Path(file).stem] = tp[Path(file).stem].astype('float64')
    tp.drop(tp[tp['NDVI'] < 0.2].index, inplace=True)
    
    return tp

In [160]:
# run
up = NDVI02(up, NDVIup_file, NDVIup_src)
down = NDVI02(down, NDVIdown_file, NDVIdown_src)
br = NDVI02(br, NDVIbr_file, NDVIbr_src)

In [ ]:
## result: ['geo','subclass', 'NDVI'] || up(16979,3) down(15042,3) br(24028,3)

In [164]:
# Point Sampling
def PointSampling(file, src, tp, feature):
    WIDTH = 0.00000599299783809013749338974087784241
    HEIGHT = 0.0000059299783809007439469768700121737
    coord_list0= [(x,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list2= [(x, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list4= [(x+WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list6= [(x,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list8= [(x-WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list = [coord_list0, coord_list1, coord_list2, coord_list3, coord_list4, coord_list5, coord_list6, coord_list7, coord_list8]
    
    for i in range(9):
        globals()[f'tp{i}'] = tp
    
    for i in range(9):
        globals()[f'tp{i}'][Path(file).stem] = [x for x in src.sample(coord_list[i])] # 포인트 샘플링
        globals()[f'tp{i}'][Path(file).stem] = globals()[f'tp{i}'][Path(file).stem].astype('float64')
        globals()[f'tp{i}'].rename(columns = {feature : feature+str(i)} , inplace=True)

In [165]:
## 3by3 PS 한번에 돌리려고 했는데 모르겠음
# for i in ['NDVI', 'SBI', 'GVI', 'YVI', 'WBI']:
#     for j in [up, down, br]:
#         PointSampling(globals()[f'{i}{j}_file'], globals()[f'{i}{j}_src'], globals()[f'{j}'])

In [166]:
## run 3by3 PS
# up
PointSampling(NDVIup_file, NDVIup_src, up, 'NDVI')
PointSampling(SBIup_file, SBIup_src, up, 'SBI')
PointSampling(GVIup_file, GVIup_src, up, 'GVI')
PointSampling(YVIup_file, YVIup_src, up, 'YVI')
PointSampling(WBIup_file, WBIup_src, up, 'WBI')
# down
PointSampling(NDVIdown_file, NDVIdown_src, down, 'NDVI')
PointSampling(SBIdown_file, SBIdown_src, down, 'SBI')
PointSampling(GVIdown_file, GVIdown_src, down, 'GVI')
PointSampling(YVIdown_file, YVIdown_src, down, 'YVI')
PointSampling(WBIdown_file, WBIdown_src, down, 'WBI')
# br
PointSampling(NDVIbr_file, NDVIbr_src, br, 'NDVI')
PointSampling(SBIbr_file, SBIbr_src, br, 'SBI')
PointSampling(GVIbr_file, GVIbr_src, br, 'GVI')
PointSampling(YVIbr_file, YVIbr_src, br, 'YVI')
PointSampling(WBIbr_file, WBIbr_src, br, 'WBI')

In [167]:
print(up.columns, down.columns, br.columns)
## result: [출력결과 확인] || up(16979,47) down(15042,47) br(24028,47)

Index(['geometry', 'subclass', 'NDVI0', 'NDVI1', 'NDVI2', 'NDVI3', 'NDVI4',
       'NDVI5', 'NDVI6', 'NDVI7', 'NDVI8', 'SBI0', 'SBI1', 'SBI2', 'SBI3',
       'SBI4', 'SBI5', 'SBI6', 'SBI7', 'SBI8', 'GVI0', 'GVI1', 'GVI2', 'GVI3',
       'GVI4', 'GVI5', 'GVI6', 'GVI7', 'GVI8', 'YVI0', 'YVI1', 'YVI2', 'YVI3',
       'YVI4', 'YVI5', 'YVI6', 'YVI7', 'YVI8', 'WBI0', 'WBI1', 'WBI2', 'WBI3',
       'WBI4', 'WBI5', 'WBI6', 'WBI7', 'WBI8'],
      dtype='object') Index(['geometry', 'subclass', 'NDVI0', 'NDVI1', 'NDVI2', 'NDVI3', 'NDVI4',
       'NDVI5', 'NDVI6', 'NDVI7', 'NDVI8', 'SBI0', 'SBI1', 'SBI2', 'SBI3',
       'SBI4', 'SBI5', 'SBI6', 'SBI7', 'SBI8', 'GVI0', 'GVI1', 'GVI2', 'GVI3',
       'GVI4', 'GVI5', 'GVI6', 'GVI7', 'GVI8', 'YVI0', 'YVI1', 'YVI2', 'YVI3',
       'YVI4', 'YVI5', 'YVI6', 'YVI7', 'YVI8', 'WBI0', 'WBI1', 'WBI2', 'WBI3',
       'WBI4', 'WBI5', 'WBI6', 'WBI7', 'WBI8'],
      dtype='object') Index(['geometry', 'subclass', 'NDVI0', 'NDVI1', 'NDVI2', 'NDVI3', 'NDVI4',
       '

In [171]:
def NDVIkn02(tp):
    for i in range(9):
        tp.drop(tp[tp['NDVI'+str(i)]<0.2].index, inplace=True)
    return tp

In [172]:
up = NDVIkn02(up)
down = NDVIkn02(down)
br = NDVIkn02(br)

In [ ]:
## result: [같음] || up(15057,47) down(13409,47) br(21971,47)

In [176]:
tpall = gpd.GeoDataFrame( pd.concat( [up, down, br], ignore_index=True) )
tpall.drop(['geometry'], axis = 1, inplace=True)

In [177]:
tpall
# result: 50437 rows × 46 columns

,subclass,NDVI0,NDVI1,NDVI2,NDVI3,NDVI4,NDVI5,NDVI6,NDVI7,NDVI8,...,YVI8,WBI0,WBI1,WBI2,WBI3,WBI4,WBI5,WBI6,WBI7,WBI8
0,1,0.464282,0.409306,0.407115,0.452619,0.503392,0.578829,0.573110,0.534801,0.476644,...,-0.033923,-0.022459,-0.024248,-0.022905,-0.022820,-0.022372,-0.017260,-0.017881,-0.019022,-0.021259
1,1,0.455428,0.372647,0.296689,0.483809,0.480325,0.511838,0.552093,0.506353,0.448599,...,-0.034616,-0.013422,-0.021045,-0.037717,-0.005955,-0.013784,-0.020056,-0.014982,-0.015727,-0.017727
2,1,0.440562,0.363137,0.367951,0.365687,0.430934,0.542444,0.492815,0.491323,0.444831,...,-0.023186,-0.018089,-0.025330,-0.022900,-0.020967,-0.018136,-0.013781,-0.016303,-0.016658,-0.018071
3,2,0.496688,0.488009,0.490608,0.464490,0.480377,0.493586,0.506200,0.522930,0.508500,...,-0.018149,-0.015623,-0.013704,-0.015020,-0.015814,-0.015718,-0.015725,-0.015517,-0.014350,-0.015489
4,1,0.499544,0.397492,0.445793,0.430574,0.491367,0.536105,0.540554,0.543937,0.525552,...,-0.020858,-0.018650,-0.022599,-0.020606,-0.021052,-0.018655,-0.018037,-0.017317,-0.016331,-0.015664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50432,3,0.405451,0.430052,0.421881,0.422427,0.414712,0.400485,0.397982,0.391079,0.421407,...,-0.012133,-0.020185,-0.015561,-0.017658,-0.018128,-0.019007,-0.021131,-0.022012,-0.023889,-0.017182
50433,1,0.367431,0.339087,0.347499,0.347499,0.374962,0.396406,0.426638,0.431339,0.399603,...,-0.017424,-0.024442,-0.024315,-0.014896,-0.014896,-0.021120,-0.021162,-0.021120,-0.021670,-0.023522
50434,1,0.542233,0.538929,0.538929,0.571726,0.571170,0.564920,0.552740,0.552740,0.542233,...,-0.015747,-0.012968,-0.013850,-0.013850,-0.010441,-0.010197,-0.010661,-0.011407,-0.011407,-0.012968
50435,2,0.232292,0.231822,0.240183,0.245211,0.231559,0.221738,0.221738,0.229638,0.227073,...,-0.015426,-0.022145,-0.026387,-0.022933,-0.023956,-0.025511,-0.021392,-0.021392,-0.013672,-0.019886


In [178]:
tpall.to_csv('all_training.csv', index=False)

## 분류항목 개수 체크

In [182]:
for i in range(1,7):
    globals()[f'sc{i}'] = tpall[tpall['subclass'] == str(i)]

In [183]:
sc1

,subclass,NDVI0,NDVI1,NDVI2,NDVI3,NDVI4,NDVI5,NDVI6,NDVI7,NDVI8,...,YVI8,WBI0,WBI1,WBI2,WBI3,WBI4,WBI5,WBI6,WBI7,WBI8
0,1,0.464282,0.409306,0.407115,0.452619,0.503392,0.578829,0.573110,0.534801,0.476644,...,-0.033923,-0.022459,-0.024248,-0.022905,-0.022820,-0.022372,-0.017260,-0.017881,-0.019022,-0.021259
1,1,0.455428,0.372647,0.296689,0.483809,0.480325,0.511838,0.552093,0.506353,0.448599,...,-0.034616,-0.013422,-0.021045,-0.037717,-0.005955,-0.013784,-0.020056,-0.014982,-0.015727,-0.017727
2,1,0.440562,0.363137,0.367951,0.365687,0.430934,0.542444,0.492815,0.491323,0.444831,...,-0.023186,-0.018089,-0.025330,-0.022900,-0.020967,-0.018136,-0.013781,-0.016303,-0.016658,-0.018071
4,1,0.499544,0.397492,0.445793,0.430574,0.491367,0.536105,0.540554,0.543937,0.525552,...,-0.020858,-0.018650,-0.022599,-0.020606,-0.021052,-0.018655,-0.018037,-0.017317,-0.016331,-0.015664
6,1,0.453509,0.361856,0.392422,0.344105,0.453509,0.509236,0.537672,0.546609,0.432571,...,-0.029656,-0.018477,-0.020474,-0.021279,-0.027466,-0.018477,-0.015709,-0.012700,-0.011532,-0.016672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50417,1,0.479225,0.471335,0.471335,0.486925,0.496159,0.496708,0.487305,0.514158,0.476134,...,-0.018463,-0.012422,-0.014171,-0.014171,-0.011176,-0.009458,-0.008915,-0.009935,-0.007695,-0.013177
50427,1,0.604787,0.588897,0.574838,0.614462,0.651343,0.671010,0.631330,0.673329,0.637094,...,-0.014780,-0.010090,-0.011239,-0.011755,-0.006466,-0.004162,-0.003280,-0.008418,-0.005534,-0.007852
50431,1,0.379990,0.346344,0.411512,0.411512,0.397139,0.392601,0.419458,0.440584,0.399891,...,-0.029543,-0.019273,-0.021341,-0.005863,-0.005863,-0.011147,-0.016301,-0.015649,-0.014943,-0.019751
50433,1,0.367431,0.339087,0.347499,0.347499,0.374962,0.396406,0.426638,0.431339,0.399603,...,-0.017424,-0.024442,-0.024315,-0.014896,-0.014896,-0.021120,-0.021162,-0.021120,-0.021670,-0.023522


In [185]:
print(sc1.shape, sc2.shape, sc3.shape, sc4.shape, sc5.shape, sc6.shape)

(19850, 46) (22804, 46) (1124, 46) (3153, 46) (304, 46) (3202, 46)


## specturm analysis

In [187]:
from spectrum import Periodogram, data_cosine

DistributionNotFound: The 'easydev' distribution was not found and is required by spectrum